In [2]:
import pyaudio

def list_audio_devices():
    """
    List all available audio devices using PyAudio.

    This function initializes PyAudio, retrieves the list of audio devices,
    and prints their names. It also includes error handling to ensure proper
    cleanup of resources.
    """
    try:
        p = pyaudio.PyAudio()
        print("Available audio devices:")
        for ii in range(p.get_device_count()):
            device_name = p.get_device_info_by_index(ii).get('name')
            print(f"{ii}: {device_name}")
    except Exception as e:
        print(f"An error occurred while listing audio devices: {e}")
    finally:
        # Ensure PyAudio resources are released
        if 'p' in locals():
            p.terminate()

# Call the function to list audio devices
list_audio_devices()

Available audio devices:
0: Microsoft Sound Mapper - Input
1: Microphone (Lumina Camera - Raw
2: Surface Stereo Microphones (Sur
3: Microsoft Sound Mapper - Output
4: Surface Omnisonic Speakers (Sur
5: Speakers (Dell USB Audio)
6: Primary Sound Capture Driver
7: Microphone (Lumina Camera - Raw)
8: Surface Stereo Microphones (Surface High Definition Audio)
9: Primary Sound Driver
10: Surface Omnisonic Speakers (Surface High Definition Audio)
11: Speakers (Dell USB Audio)
12: Speakers (Dell USB Audio)
13: Surface Omnisonic Speakers (Surface High Definition Audio)
14: Surface Stereo Microphones (Surface High Definition Audio)
15: Microphone (Lumina Camera - Raw)
16: Headphones ()
17: Speakers (Dell USB Audio)
18: Microphone (Dell USB Audio)
19: Headset (@System32\drivers\bthhfenum.sys,#2;%1 Hands-Free%0
;(Shiva’s AirPods Pro #2 - Find My))
20: Headset (@System32\drivers\bthhfenum.sys,#2;%1 Hands-Free%0
;(Shiva’s AirPods Pro #2 - Find My))
21: Headphones 1 (Realtek HD Audio 2nd output with

In [5]:
import pyaudio
import wave

def test_microphone():
    """
    Test the microphone by recording audio and playing it back.

    This function captures audio from the default input device (microphone),
    saves it to a temporary WAV file, and plays it back to ensure the microphone
    is working correctly.
    """
    # Audio configuration
    chunk = 1024  # Number of frames per buffer
    format = pyaudio.paInt16  # 16-bit audio format
    channels = 1  # Mono audio
    rate = 44100  # Sampling rate (44.1 kHz)
    record_seconds = 5  # Duration of the recording
    output_filename = "test_audio.wav"

    # Initialize PyAudio
    p = pyaudio.PyAudio()

    try:
        # Open the microphone stream
        print("Recording...")
        stream = p.open(format=format,
                        channels=channels,
                        rate=rate,
                        input=True,
                        frames_per_buffer=chunk)

        frames = []

        # Record audio in chunks
        for _ in range(0, int(rate / chunk * record_seconds)):
            data = stream.read(chunk)
            frames.append(data)

        print("Recording complete. Saving audio...")

        # Save the recorded audio to a WAV file
        with wave.open(output_filename, 'wb') as wf:
            wf.setnchannels(channels)
            wf.setsampwidth(p.get_sample_size(format))
            wf.setframerate(rate)
            wf.writeframes(b''.join(frames))

        print(f"Audio saved to {output_filename}. Playing back...")

        # Play back the recorded audio
        stream.stop_stream()
        stream.close()

        # Open the WAV file for playback
        wf = wave.open(output_filename, 'rb')
        playback_stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
                                 channels=wf.getnchannels(),
                                 rate=wf.getframerate(),
                                 output=True)

        # Read and play audio data
        data = wf.readframes(chunk)
        while data:
            playback_stream.write(data)
            data = wf.readframes(chunk)

        playback_stream.stop_stream()
        playback_stream.close()

        print("Playback complete.")

    except Exception as e:
        print(f"An error occurred: {e}")

    finally:
        # Terminate PyAudio
        p.terminate()

# Run the microphone test
test_microphone()

Recording...
Recording complete. Saving audio...
Audio saved to test_audio.wav. Playing back...
Playback complete.


In [6]:
import logging 
import os
# set the directory to the location of the script
try:
    target_directory = os.getenv("TARGET_DIRECTORY", os.getcwd())  # Use environment variable if available
    if os.path.exists(target_directory):
        os.chdir(target_directory)
        logging.info(f"Successfully changed directory to: {os.getcwd()}")
    else:
        logging.error(f"Directory does not exist: {target_directory}")
except Exception as e:
    logging.exception(f"An error occurred while changing directory: {e}")

In [7]:
from src.realtime_agent.ws_manager import WSManager

In [ ]:
system_prompt = "You are a helpful AI assistant. You always answer immediately to anything you hear."

manager = WSManager(system_prompt=system_prompt)
manager.start()

Connecting to wss://aoai-ai-factory-eus2-dev.openai.azure.com/openai/realtime?api-version=2024-10-01-preview&deployment=gpt-4o-realtime-preview-2022-12-17&api-key=c7201c31982f45bfbc25d93acfdcdd3e


🎙️ Mic Input: █                                                 